In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
f=open('/content/drive/My Drive/file.txt')

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [5]:
x=[]
y=[]
bad_chars = [';', ':', '!', "*","@","#","$","%","^","&","-","(",")","_",",",".","?"] 
for l in f:
  for k in bad_chars:
    l.replace(k," ")
  t1=l.split()
  y.append(int(t1.pop()))
  t=[]
  for word in t1:
    if word not in stop_words:
      t.append(word)
  
  x.append(t)
  
for i in range(len(x)):
  for j in range(len(x[i])):
    x[i][j]=x[i][j].lower()

# **Finding vocab from overall data**

In [6]:
unique=[]
for i in range(len(x)):
  unique=set(unique).union(set(x[i]))
print(len(unique))


3158


# **Splitting data into training and testing**

In [7]:
x_train=[]
x_test=[]
y_train=[]
y_test=[]
for i in range(len(x)):
  if(i<750):
    x_train.append(x[i])
    y_train.append(y[i])
  else:
    x_test.append(x[i])
    y_test.append(y[i])


In [8]:
len(x_test)

250

In [9]:
train=[]
for i in range(len(x_train)):
  d=dict.fromkeys(unique, 0) 
  for words in x_train[i]:
    d[words]+=1
  train.append(d)
  

In [10]:
test=[]
for i in range(len(x_test)):
  d=dict.fromkeys(unique, 0) 
  for words in x_test[i]:
    d[words]+=1
  test.append(d)

# **function for calculation IDF**

In [11]:
import math
def computeIDF(L,unique):
  idf = dict.fromkeys(unique, 0)
  for i in range(len(L)):
    for word in L[i].keys():
      if  L[i][word]>0:
        idf[word]+=1
  N=len(L)
  for key in idf.keys():
    idf[key]=math.log10(N / float(idf[key]+1))
  return idf

# **Computing IDF from test data**

In [12]:
idf=computeIDF(train,unique)

# **Function for calculating TFIDF**

In [13]:
def computeTFIDF(L,IDF):
  for i in range(len(L)):
    for words in L[i].keys():
      L[i][words]=L[i][words]/len(x[i])*IDF[words]
  return L


# **TFIDF for test and train**

In [14]:
tfidf_train=computeTFIDF(train,idf)     
tfidf_test=computeTFIDF(test,idf)

In [15]:
len(tfidf_train)

750

In [16]:
import pandas as pd
df_train=pd.DataFrame(tfidf_train)
df_test=pd.DataFrame(tfidf_test)


# **PCA**

In [17]:
from sklearn.decomposition import PCA
pca=PCA(n_components=10)
pca.fit(df_train)
#pca on train
pca_train=pca.transform(df_train)

#pca on test 
pca_test=pca.transform(df_test)

In [18]:
pca_train.shape
print(len(y_train))

750


# **SVM implementation**

In [19]:
#LINEAR KERNEL
from sklearn import svm
linear_svm = svm.SVC(C=500,kernel="linear")
linear_svm.fit(pca_train,y_train)

#accuracy on train data
print("Linear SVM train accuracy:",linear_svm.score(pca_train,y_train))

#accuracy on test data
print("Linear SVM train accuracy:",linear_svm.score(pca_test,y_test))

#number of Support Vector
print("Number of SV:",linear_svm.support_vectors_.shape[0])


Linear SVM train accuracy: 0.6186666666666667
Linear SVM train accuracy: 0.496
Number of SV: 637


In [20]:
#POLY KERNEL
from sklearn import svm
poly_svm = svm.SVC(C=500,kernel="poly")
poly_svm.fit(pca_train,y_train)

#accuracy on train data
print("poly SVM train accuracy:",poly_svm.score(pca_train,y_train))

#accuracy on test data
print("poly SVM train accuracy:",poly_svm.score(pca_test,y_test))

#number of Support Vector
print("Number of SV:",poly_svm.support_vectors_.shape[0])


poly SVM train accuracy: 0.5853333333333334
poly SVM train accuracy: 0.408
Number of SV: 665


In [22]:
#RBF KERNEL
from sklearn import svm
rbf_svm = svm.SVC(C=500,kernel="rbf")
rbf_svm.fit(pca_train,y_train)
#accuracy on train data
print("rbf SVM train accuracy:",rbf_svm.score(pca_train,y_train))

#accuracy on test data
print("rbf SVM test accuracy:",rbf_svm.score(pca_test,y_test))

#number of Support Vector
print("Number of SV:",rbf_svm.support_vectors_.shape[0])





rbf SVM train accuracy: 0.66
rbf SVM test accuracy: 0.464
Number of SV: 597
